# Building a Neural Network from Scratch

https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65 

### Abstract Base Class : Layer
The abstract class Layer, which all other layers will inherit from, handles simple properties which are an input, an output, and both a forward and backward methods.

In [1]:

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In the abstract class above, backward_propagation function has an extra parameter, learning_rate, which is controlling the amount of learning/updating parameters using gradient descent.

### Backward Propagation
Suppose we have a matrix containing the derivative of the error with respect to that layer’s output: $\frac{\partial E}{\partial Y}$

We need :
- The derivative of the error with respect to the parameters ($\frac{\partial E}{\partial W}$, $\frac{\partial E}{\partial B}$)
- The derivative of the error with respect to the input ($\frac{\partial E}{\partial X}$)

Let's calculate $\frac{\partial E}{\partial W}$. This matrix should be the same size as $W$ itself : 

$i x j$ where $i$ is the number of input neurons and $j$ the number of output neurons. We need one gradient for every weight

### Coding the Fully Connected Layer

In [2]:
#from layer import Layer
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of edges that connects to neurons in next layer
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

### Activation Layer
All the calculation we did until now were completely linear, may not learn well. We need to add non-linearity to the model by applying non-linear functions to the output of some layers.

Now we need to redo the whole process for this new type of layer!

In [3]:
#from layer import Layer

# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

You can also write some activation functions and their derivatives in a separate file. These will be used later to create an ActivationLayer.

In [4]:
import numpy as np

# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

### Loss Function
Until now, for a given layer, we supposed that ∂E/∂Y was given (by the next layer). But what happens to the last layer? How does it get ∂E/∂Y? We simply give it manually, and it depends on how we define the error.
The error of the network, which measures how good or bad the network did for a given input data, is defined by you. 

There are many ways to define the error, and one of the most known is called MSE — Mean Squared Error.

In [5]:

import numpy as np

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

### Network Class
Almost done ! We are going to make a Network class to create neural networks very easily using the building blocks we have prepared so far.


In [6]:
# example of a function for calculating softmax for a list of numbers
from numpy import exp
 
# calculate the softmax of a vector
def softmax(vector):
    e = exp(vector)
    return e / e.sum()

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

        
    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network 
    
    def fit(self, x_train, y_train, epochs, learning_rate):
        '''
        Fit function does the training. 
        Training data is passed 1-by-1 through the network layers during forward propagation.
        Loss (error) is calculated for each input and back propagation is performed via partial 
        derivatives on each layer.
        '''
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

### Building Neural Networks
Finally ! We can use our class to create a neural network with as many layers as we want ! We are going to build two neural networks : a simple XOR and a MNIST solver.


### Solve XOR
Starting with XOR is always important as it’s a simple way to tell if the network is learning anything at all.

In [8]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.296054
epoch 2/1000   error=0.295202
epoch 3/1000   error=0.294748
epoch 4/1000   error=0.294448
epoch 5/1000   error=0.294217
epoch 6/1000   error=0.294020
epoch 7/1000   error=0.293839
epoch 8/1000   error=0.293669
epoch 9/1000   error=0.293504
epoch 10/1000   error=0.293344
epoch 11/1000   error=0.293187
epoch 12/1000   error=0.293032
epoch 13/1000   error=0.292880
epoch 14/1000   error=0.292729
epoch 15/1000   error=0.292580
epoch 16/1000   error=0.292433
epoch 17/1000   error=0.292288
epoch 18/1000   error=0.292143
epoch 19/1000   error=0.292000
epoch 20/1000   error=0.291859
epoch 21/1000   error=0.291718
epoch 22/1000   error=0.291578
epoch 23/1000   error=0.291440
epoch 24/1000   error=0.291302
epoch 25/1000   error=0.291165
epoch 26/1000   error=0.291028
epoch 27/1000   error=0.290892
epoch 28/1000   error=0.290757
epoch 29/1000   error=0.290623
epoch 30/1000   error=0.290488
epoch 31/1000   error=0.290355
epoch 32/1000   error=0.290221
epoch 33/1000   e

### Solve MNIST
We didn’t implemented the Convolutional Layer but this is not a problem. 
All we need to do is to reshape our data so that it can fit into a Fully Connected Layer.
MNIST Dataset consists of images of digits from 0 to 9, of shape 28x28x1. 
The goal is to predict what digit is drawn on a picture.

In [9]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

epoch 1/35   error=0.247857
epoch 2/35   error=0.116482
epoch 3/35   error=0.094183
epoch 4/35   error=0.080750
epoch 5/35   error=0.069387
epoch 6/35   error=0.059157
epoch 7/35   error=0.050734
epoch 8/35   error=0.044025
epoch 9/35   error=0.038943
epoch 10/35   error=0.034187
epoch 11/35   error=0.030527
epoch 12/35   error=0.027398
epoch 13/35   error=0.025067
epoch 14/35   error=0.022808
epoch 15/35   error=0.021122
epoch 16/35   error=0.019502
epoch 17/35   error=0.018085
epoch 18/35   error=0.016745
epoch 19/35   error=0.015536
epoch 20/35   error=0.014561
epoch 21/35   error=0.013665
epoch 22/35   error=0.012926
epoch 23/35   error=0.012193
epoch 24/35   error=0.011492
epoch 25/35   error=0.010931
epoch 26/35   error=0.010486
epoch 27/35   error=0.010071
epoch 28/35   error=0.009697
epoch 29/35   error=0.009347
epoch 30/35   error=0.009079
epoch 31/35   error=0.008784
epoch 32/35   error=0.008544
epoch 33/35   error=0.008240
epoch 34/35   error=0.008085
epoch 35/35   error=0.0

In [10]:
print(x_test[0:3])

[[[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]]


In [11]:
import numpy as np 

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2
def relu(x):
    return np.maximum(x, 0)
def relu_prime(x):
    return np.array(x >= 0).astype('int')

In [12]:
x = 1.0
print('Applying Sigmoid Activation on (%.1f) gives %.1f' % (x, sigmoid(x)))

x = -10.0
print('Applying Sigmoid Activation on (%.1f) gives %.1f' % (x, sigmoid(x)))

x = 0.0
print('Applying Sigmoid Activation on (%.1f) gives %.1f' % (x, sigmoid(x)))

x = 15.0
print('Applying Sigmoid Activation on (%.1f) gives %.1f' % (x, sigmoid(x)))

x = -2.0
print('Applying Sigmoid Activation on (%.1f) gives %.1f' % (x, sigmoid(x)))


Applying Sigmoid Activation on (1.0) gives 0.7
Applying Sigmoid Activation on (-10.0) gives 0.0
Applying Sigmoid Activation on (0.0) gives 0.5
Applying Sigmoid Activation on (15.0) gives 1.0
Applying Sigmoid Activation on (-2.0) gives 0.1


In [13]:
x = 1.0
print('Applying Relu on (%.1f) gives %.1f' % (x, relu(x)))
x = -10.0
print('Applying Relu on (%.1f) gives %.1f' % (x, relu(x)))
x = 0.0
print('Applying Relu on (%.1f) gives %.1f' % (x, relu(x)))
x = 15.0
print('Applying Relu on (%.1f) gives %.1f' % (x, relu(x)))
x = -20.0
print('Applying Relu on (%.1f) gives %.1f' % (x, relu(x)))

Applying Relu on (1.0) gives 1.0
Applying Relu on (-10.0) gives 0.0
Applying Relu on (0.0) gives 0.0
Applying Relu on (15.0) gives 15.0
Applying Relu on (-20.0) gives 0.0


In [14]:
from sklearn.metrics import accuracy_score
y_pred = [1, 5, 7, 8]
y_true = [6, 5, 2, 3]
accuracy_score(y_true, y_pred)

0.25

In [15]:
accuracy_score(y_true, y_pred, normalize=False)

1

In [16]:
import numpy as np
accuracy_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2)))

0.5

In [17]:
from sklearn.metrics import classification_report

def evaluate(x_t, y_t):
  true = [np.argmax(y) for x, y in zip(x_t, y_t)]
  pred = [np.argmax(net.predict(x)[0]) for x, y in zip(x_t, y_t)]
  print(classification_report(true, pred))
evaluate(x_test, y_test)

              precision    recall  f1-score   support

           0       0.93      0.88      0.91       980
           1       0.96      0.96      0.96      1135
           2       0.87      0.76      0.81      1032
           3       0.86      0.70      0.77      1010
           4       0.79      0.72      0.75       982
           5       0.54      0.74      0.62       892
           6       0.87      0.78      0.82       958
           7       0.82      0.87      0.84      1028
           8       0.66      0.74      0.69       974
           9       0.70      0.74      0.72      1009

    accuracy                           0.79     10000
   macro avg       0.80      0.79      0.79     10000
weighted avg       0.80      0.79      0.79     10000



In [18]:
import pandas as pd
myDataSet = pd.read_csv(r"C:\Users\User\Desktop\diabetes.csv")

In [19]:
myDataSet.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [20]:
myDataSet.var().round(3)
myDataSet.rename(columns={'Diabete': 'Pregnancies'}, inplace=True)

x = myDataSet.drop(columns='Pregnancies')
y = myDataSet['Outcome']

In [21]:

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# load MNIST from server
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

x_train.var().round(3)

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.values.reshape(x_train.shape[0], 1, 8)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.values.reshape(x_test.shape[0],1,8)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(8, 100))                
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.add(FCLayer(100, 50))                   
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.add(FCLayer(50, 2))                   
net.add(ActivationLayer(sigmoid, sigmoid_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

epoch 1/35   error=0.231594
epoch 2/35   error=0.229309
epoch 3/35   error=0.227885
epoch 4/35   error=0.226556
epoch 5/35   error=0.225272
epoch 6/35   error=0.224004
epoch 7/35   error=0.222739
epoch 8/35   error=0.221465
epoch 9/35   error=0.220170
epoch 10/35   error=0.218829
epoch 11/35   error=0.217415
epoch 12/35   error=0.215897
epoch 13/35   error=0.214241
epoch 14/35   error=0.212408
epoch 15/35   error=0.210356
epoch 16/35   error=0.208049
epoch 17/35   error=0.205461
epoch 18/35   error=0.202594
epoch 19/35   error=0.199498
epoch 20/35   error=0.196283
epoch 21/35   error=0.193113
epoch 22/35   error=0.190164
epoch 23/35   error=0.187568
epoch 24/35   error=0.185381
epoch 25/35   error=0.183588
epoch 26/35   error=0.182134
epoch 27/35   error=0.180949
epoch 28/35   error=0.179970
epoch 29/35   error=0.179143
epoch 30/35   error=0.178431
epoch 31/35   error=0.177805
epoch 32/35   error=0.177248
epoch 33/35   error=0.176747
epoch 34/35   error=0.176291
epoch 35/35   error=0.1

In [22]:
evaluate(x_test, y_test)

              precision    recall  f1-score   support

           0       0.80      0.65      0.72       100
           1       0.52      0.70      0.60        54

    accuracy                           0.67       154
   macro avg       0.66      0.68      0.66       154
weighted avg       0.70      0.67      0.68       154

